**Scenario - 1 :**

Hope, you are getting more and more comfortable working with Series and Dataframes. Now, since the data quality checks are taken care of, It's time we perform some real world data wrangling to generate some reports/insights etc.

Check the code snippets below where we are reading customers, orders and transactions data. Click [here](https://raw.githubusercontent.com/mentorskool/python-essentials/main/Module-6-PY-Learning-4/Data/globalmart-business-data.xlsx) to download the Excel sheet containing the datasets

You are required to answer the below questions :      
* How many customers belong to Corporate segment?
* How many customers from Corporate segment also belong to the outlook email domain?
* Report segment wise count of customers. Which segment has highest number of customers?
* Which segment of customers has placed the highest number of orders?
* Prepare a summary to show month-year wise (based on order purchase date) orders delivered by different ship modes for the year 2017

Expected Outcome :  

![](https://msklbusinessdata.blob.core.windows.net/python-masterclass/12-pivot-01.png)   

* Enhance the same report to show the ship modes under various Customer segments

Expected Outcome :

![](https://msklbusinessdata.blob.core.windows.net/python-masterclass/12-pivot-02.png)



**Important Note :**

We are going to perform familiar data operations like Filtering, Sorting, Merging (Excel like Vlookup or Joins in SQL), Group-Aggregation, Pivoting etc.

You must have already done these tasks while working with data in Excel or SQL

Let's see how Python performs these tasks using the famous Pandas library

In [1]:
# Let's import required libraries
import pandas as pd
import numpy as np

In [2]:
# reading customers data into a new dataframe customers

customers = pd.read_excel('https://raw.githubusercontent.com/mentorskool/python-essentials/main/Module-6-PY-Learning-4/Data/globalmart-business-data.xlsx', sheet_name = 'customers')

# reading orders data into a new dataframe orders

orders = pd.read_excel('https://raw.githubusercontent.com/mentorskool/python-essentials/main/Module-6-PY-Learning-4/Data/globalmart-business-data.xlsx', sheet_name = 'orders')

# reading transactions data into a new dataframe transactions

transactions = pd.read_excel('https://raw.githubusercontent.com/mentorskool/python-essentials/main/Module-6-PY-Learning-4/Data/globalmart-business-data.xlsx', sheet_name = 'transactions')



In [3]:
# also we can load all sheets into one, and then one by one access each sheet from that
# globalmart_data = pd.read_excel('https://raw.githubusercontent.com/mentorskool/python-essentials/main/Module-6-PY-Learning-4/Data/globalmart-business-data.xlsx', sheet_name=None)

# customers = globalmart_data['customers']

# orders = globalmart_data['orders']

# transactions = globalmart_data['transactions']

In [ ]:
# Check the top 5 rows in customers dataframe

customers.head()

In [ ]:
# Check the top 5 rows in orders dataframe

orders.head()

In [ ]:
# Check the top 5 rows in transactions dataframe

transactions.head()

### **Task 1** - How many customers belong to Corporate segment?

In [ ]:
# Filter column named segment for values = 'Corporate'
# Output will be dataframe with only those records where segment = 'Corporate'

customers[customers['segment'] == 'Corporate']

In [ ]:
# Count all records which satisfy the condition stated above

len(customers[customers['segment'] == 'Corporate'])

### **Task 2** - How many customers from Corporate segment also belong to the outlook email domain?

In [ ]:
# Split customer_email column on the character @ and fetch the part on the right of '@'
# Store the fetched part into a new column called 'email_domain'

customers['email_domain'] = customers['customer_email'].str.split('@',expand = True)[1].str.split('.',expand = True)[0]
customers.head()

In [ ]:
# Now fetch records from cutomers dataframe only those records where segment = 'Corporate'
# And email_domain = 'outlook'

customers[(customers['segment'] == 'Corporate') & (customers['email_domain'] == 'outlook')]

In [ ]:
# Use len to fetch the count of records matching the condition stated above

len(customers[(customers['segment'] == 'Corporate') & (customers['email_domain'] == 'outlook')])

### **Task 3** - Report segment wise count of customers. Which segment has highest number of customers?

In [ ]:
# Approach 1

customers['segment'].value_counts()

# Note how the output looks similar to group by and aggregate(count)

In [ ]:
# Approach 2

customers.groupby(['segment'])['customer_id'].count().reset_index()

# Note how similar is the syntax to a typical group-by statement in SQL
# Check the snippers : .groupby() and .count()
# Why reset_index()?
# Try once without reset_index() and check if the output is any different

### **Task 4** - Which segment of customers has placed the highest number of orders?

You have to merge two tables together: Customer and Order

How do you achieve the same in SQL?
* By making use of JOINS

How do you achieve the same in Excel?
* By making use of VLOOKUP() function


Here customer_id of df1 (customer_detalis) is the common key , by which we can combine two dataframes

Before go deep into code, let's explore this documentation : https://pandas.pydata.org/docs/user_guide/merging.html

Did you figure out different ways to achieve merging of data?

Below are three key methods :

* Concatenate
  * Just combining dataframes
  * Index is not reset
  * Default axis is 0
* Merge
  * pandas has full-featured, high performance in-memory join operations idiomatically very similar to relational databases like SQL.
  * These methods perform significantly better (in some cases well over an order of magnitude better) than other open source implementations (like base::merge.data.frame in R).
  * The reason for this is careful algorithmic design and the internal layout of the data in DataFrame.
* Join
  * This is a convenient method for combining the columns DataFrames into a single result DataFrame



In [ ]:
# Approach 1
# Put the dataframes customers and orders in a list and pass the list in the concat() method
# concat() method looks for a column with matching names in both dataframes
# Seems customer_id is the one having exactly matching name. Hence, it qualifies as the common key

pd.concat([customers,orders])

Did you observe so many NaN values in the output dataframe?

The primary reason of this being the nature of joins. Go back to your SQL skills and recall how left joins, right joins and full joins lead to lot of null values from the left, right or both tables

What if we are only concerned about records with matching customer_ids. This would need an inner join to be performed.

Let's check the code below to perform inner join in Python

In [ ]:
# Approach 2
pd.concat([customers,orders], axis = 1, join = 'inner')

# Figure out the reason why axis = 1 is important to add here. Experiment without axis = 1 and see if that impacts the code in anyway

But we are loosing some information of reviews data

But by default concat have *two type of join* : **inner and outer**

#### Type of Merges :    
In the context of merging datasets, there are different types of relationships that can occur between the merge keys in the left and right datasets. These relationships are classified as follows:

* **"One-to-One"** or **"1:1"**: This relationship type checks if the merge keys are unique in both the left and right datasets.

* **"One-to-Many"** or **"1:m"**: This relationship type checks if the merge keys are unique in the left dataset.

* **"Many-to-One"** or **"m:1"**: This relationship type checks if the merge keys are unique in the right dataset.

* **"Many-to-Many"** or **"m:m"**: This relationship type is allowed, but it does not result in any specific checks.

These relationship types help determine the uniqueness and compatibility of merge keys between datasets, ensuring the appropriate merging and data combination processes.


In [ ]:
# Approach 3
orders.merge(customers, on='customer_id', validate='one_to_one')

# Can you figure out why this code would fail?

In [ ]:
# one_to_one not possible , because customer_id is not unique in orders dataset. One customer can place multiple orders and hence customer_id can repeat
# customer: customer_id - unique
# order: customer_id - many

# Approach 4

customers.merge(orders, on='customer_id', validate='1:m')

In [ ]:
# Approach 5

orders.merge(customers, on='customer_id', validate='m:1')

* left   : LEFT OUTER JOIN    : Use whole keys from left frame and intersection of keys from right frame
* right  : RIGHT OUTER JOIN   : Use whole keys from right frame and intersection of keys from left frame
* outer  : FULL OUTER JOIN    : Use union of keys from both frames
* inner  : INNER JOIN         : Use intersection of keys from both frames : default

**Here are the commonly used join types and their behaviors:**
* **Left Outer Join**: In this type of join, all the keys from the left dataset are included in the resulting dataset, along with the intersection of keys from the right dataset.

* **Right Outer Join**: This join type is similar to the left outer join, but the keys from the right dataset are included in the resulting dataset, along with the intersection of keys from the left dataset.

* **Full Outer Join**: With a full outer join, the resulting dataset includes the union of keys from both the left and right datasets. It combines all the keys and values from both datasets, including the intersection of keys.

* **Inner Join**: The inner join is the default join type. It includes only the intersection of keys from both the left and right datasets. In other words, it includes only the keys and values that have a match in both datasets.

In [ ]:
# Check if this code is a valid one?

customers.merge(orders, on='customer_id', validate='one_to_many', how='outer')

In [ ]:
# How about this one?

customers.merge(orders, on='customer_id', validate='one_to_many', how='inner')

Strange! You never faced wrong output message like the one found with concat() method.

The reason being, there we combined two dataframes without mentioning the primary key but in the case of merge(), we mentioned the column name on which merge to take place. Hence, we are getting the correct outcome.

In [ ]:
# Approach 6

pd.merge(orders,customers,on=['customer_id'])

In [22]:
# Final merged dataframe

df = pd.merge(orders,customers,on=['customer_id'])

In [ ]:
# Filter the dataframe on segment column
# And save the output in a new variable df1
df1 = df[df['segment']=='Corporate']

# Apply group-by on customer_id column and perform count on order_id column
# Reset the index in the end and save the results back to df1 variable

df1 = df1.groupby(['customer_id'])['order_id'].count().reset_index()
df1

In [ ]:
# Finally sort the dataframe df1 on order_id in descending order (Note ascending = False means descending)

df1.sort_values('order_id', ascending=False)

### **Task 5** - Prepare a summary to show month-year wise (based on order purchase date) orders delivered by different ship modes for the year 2017

In order to attempt this task, we need to do some data preparation :      

* We need to derive a column like Jan-2017 from a column with values like '15-01-2017'
* We need to derive order_year and order_month in number format. Ex. for the value '15-01-2017', order_year will be 2017 and order_month will be 1
* We need to ensure that we consider only those orders which are delivered
* Finally, we need to sort the result from Jan-2017 to Dec-2017 ( in the order of month within an year)
* Ensure that only those orders which were placed in 2017 are considered for this report

In [ ]:
# Let's first check whether there are orders with status other than 'delivered'

df['order_status'].value_counts()

In [26]:
# Prepare a new column showing date in MMM-YYYY format
df['order_purchase_month_year'] = pd.DatetimeIndex(df['order_purchase_date']).strftime('%b-%Y')

# Pull out order_purchase_month in number format
df['order_purchase_month'] = pd.DatetimeIndex(df['order_purchase_date']).month

# Pull out order_purchase_year in number format
df['order_purchase_year'] = pd.DatetimeIndex(df['order_purchase_date']).year

# Filter all orders which are delivered
df['order_deliver']=np.where(df['order_status'] == 'delivered',1,0)

# Filter all orders which are placed in the year 2017
x=df[df['order_purchase_year'] == 2017]


Now, before we proceed, It's important to spend some time on learning how pivoting related tasks can be performed using Python.

You must have come across situations when you have to pivot data to arrive at the desired result. The report format required as part of this task is also a typical outcome of pivoting activity.

We'll highly recommend you to spend some time in [this](https://pbpython.com/pandas-pivot-table-explained.html) article which elaborates this topic quite well

In [ ]:
# Based on your study of the article, try to decode what the below code snippets are doing?

y=x.pivot_table(index=['order_purchase_month_year','order_purchase_month'],columns='ship_mode', values='order_deliver', aggfunc=np.sum)
y=y.sort_values(by='order_purchase_month')
y=y.reset_index()
y.drop(['order_purchase_month'],axis=1,inplace=True)
y.set_index('order_purchase_month_year')

### **Task 6** - Enhance the same report to show the ship modes under various Customer segments

In [ ]:
# Check how this tasks is just an enhancement to the previous task. Most of the code remains the same

y=x.pivot_table(index=['order_purchase_month_year','order_purchase_month'],columns=['segment','ship_mode'], values='order_deliver', aggfunc=np.sum)
y=y.sort_values(by='order_purchase_month')
y=y.reset_index()
y.drop(['order_purchase_month'],axis=1,inplace=True)
y.set_index('order_purchase_month_year')